# LangGraph Stream Parser - Jupyter Display Demo

This notebook demonstrates the value of `JupyterDisplay` for visualizing LangGraph agent streams.

## The Problem

When streaming from a LangGraph agent, the raw output is:
- **Verbose** - Nested dictionaries with metadata you don't care about
- **Hard to follow** - Tool calls, messages, and interrupts all mixed together
- **Not real-time friendly** - Output floods the cell instead of updating in place

## The Solution

`JupyterDisplay` provides:
- **Clean panels** - Content, tools, and interrupts in organized boxes
- **Live updates** - Updates in place instead of flooding output
- **Tool lifecycle** - See tools transition from running → success/error with timing
- **Prominent interrupts** - Human-in-the-loop requests are impossible to miss

## Setup

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from agent import agent

---
## Before: Raw Streaming Output

This is what you get with vanilla `graph.stream()` - a flood of nested dictionaries:

In [3]:
# Raw streaming - hard to follow!
for chunk in agent.stream(
    dict(messages=["What files are in the current directory?"]), 
    config=dict(configurable=dict(thread_id="raw-demo")),
    stream_mode="updates"
):
    print(chunk)
    print("---")

{'PatchToolCallsMiddleware.before_agent': {'messages': Overwrite(value=[HumanMessage(content='What files are in the current directory?', additional_kwargs={}, response_metadata={}, id='c9b8850d-d13c-4a62-b7d4-ed3232776a08')])}}
---
{'SummarizationMiddleware.before_model': None}
---{'model': {'messages': [AIMessage(content=[{'text': "I'll list the files in the current directory for you.", 'type': 'text'}, {'id': 'toolu_01VpZVL9rKCxtam4bLiodd3F', 'input': {'path': '/'}, 'name': 'ls', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01QE6LBjhUaPwnJydkqPHMoX', 'model': 'claude-sonnet-4-5-20250929', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 6258, 'input_tokens': 3, 'output_tokens': 64, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-sonnet-4-5-20250929', 'model_provider': 'anthr

---
## After: JupyterDisplay

Same stream, but with clean, live-updating panels:

In [10]:
from langgraph_stream_parser.adapters.jupyter import JupyterDisplay

In [11]:
display = JupyterDisplay()

display.stream(
    agent.stream(
        dict(messages=["What files are in the current directory?"]), 
        config=dict(configurable=dict(thread_id="display-demo")),
        stream_mode="updates"
    )
)

╭───────────────────────────────────────────── Tools ──────────────────────────────────────────────╮
│                                                                                                  │
│   Status     Tool   Args           Time                                                          │
│  ───────────────────────────────────────                                                         │
│   OK         ls     path=/         11ms                                                          │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── Assistant ────────────────────────────────────────────╮
│ The current directory contains the following files and folders:                                  │
│                                                                                                  │
│ 1. **/.ipynb_checkpoints/** - Directory for Jupyter notebook checkpoints                         │
│ 2. **/__pycache__/** - Directory for Python bytecode cache files                                 │
│ 3. **/agent.py** - Python script file                                                            │
│ 4. **/jupyter_display.py** - Python script file                                                  │
│ 5. **/jupyter_example.ipynb** - Jupyter notebook file (this is the "third one" I showed you      │
│ earlier)                                                                                         │
│                                                                                                  │
│ Would you like me to explore any of these files in more detail?                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

Stream completed

In [5]:
stop

NameError: name 'stop' is not defined

In [12]:
display = JupyterDisplay()

display.stream(
    agent.stream(
        dict(messages=["what's in the third one?"]), 
        config=dict(configurable=dict(thread_id="display-demo")),
        stream_mode="updates"
    )
)

╭───────────────────────────────────────────── Tools ──────────────────────────────────────────────╮
│                                                                                                  │
│   Status     Tool        Args                        Time                                        │
│  ─────────────────────────────────────────────────────────                                       │
│   OK         read_file   file_path=/agent.py         15ms                                        │
│                                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── Assistant ────────────────────────────────────────────╮
│ The third item in the list is **/agent.py**. Let me read it to show you what's inside:The third  │
│ file is **/agent.py**, which contains:                                                           │
│                                                                                                  │
│ **Purpose**: Creates a LangGraph agent with filesystem access and additional tools.              │
│                                                                                                  │
│ **Key components**:                                                                              │
│                                                                                                  │
│ 1. **`think_tool`** - A reflection tool that allows the agent to pause and think about its       │
│ actions and reasoning                                                                            │
│                                                                                                  │
│ 2. **`bash`** - A tool to execute bash commands with:                                            │
│    - Command execution in the workspace directory                                                │
│    - Timeout support (default 60 seconds)                                                        │
│    - Sandbox mode for security (using bubblewrap on Linux)                                       │
│    - Returns stdout, stderr, return code, and status                                             │
│                                                                                                  │
│ 3. **Agent creation** - Uses `create_deep_agent` with:                                           │
│    - FilesystemBackend with virtual mode enabled                                                 │
│    - Both `think_tool` and `bash` tools                                                          │
│    - Interrupts enabled for bash commands (requires approval before executing)                   │
│    - InMemorySaver for checkpointing                                                             │
│                                                                                                  │
│ This creates an agent that can interact with files, execute bash commands (with human approval), │
│ and reflect on its reasoning.                                                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

Stream completed

---
## Key Features

### 1. Tool Lifecycle Tracking

Tools show their status in real-time:
- `⏳ ...` - Running
- `✅ OK` - Success (with execution time)
- `❌ ERR` - Error

### 2. Interrupt Handling

When the agent needs approval (like for `bash` commands), you get a prominent red panel showing:
- The tool requesting approval
- The arguments it wants to use
- Allowed decisions (approve/reject/edit)

### 3. Content Accumulation

Assistant responses stream smoothly in a blue panel instead of chunk-by-chunk spam.

---
## Manual Event Processing

For more control, you can process events individually:

In [6]:
from langgraph_stream_parser import StreamParser
from langgraph_stream_parser.events import InterruptEvent, ContentEvent, ToolCallStartEvent

parser = StreamParser()
display = JupyterDisplay()

for event in parser.parse(
    agent.stream(
        dict(messages=["Count Python files in this directory"]),
        config=dict(configurable=dict(thread_id="manual-demo")),
        stream_mode="updates"
    )
):
    display.update(event)
    
    # You can also handle events programmatically
    if isinstance(event, InterruptEvent):
        print(f"\n[Interrupt detected - {len(event.action_requests)} action(s) need approval]")

NameError: name 'JupyterDisplay' is not defined

---
## Configuration Options

In [9]:
# Customize the display
display = JupyterDisplay(
    show_timestamps=False,     # Show/hide timestamps
    show_tool_args=True,       # Show tool arguments in the status table
    max_content_preview=200,   # Max characters for extraction previews
)

---
## Summary

| Raw Streaming | JupyterDisplay |
|---------------|----------------|
| Nested dicts flood output | Clean, organized panels |
| Tool status buried in data | Visual tool lifecycle (⏳→✅/❌) |
| Interrupts easy to miss | Prominent red interrupt panel |
| Content comes in chunks | Smooth content accumulation |
| No timing information | Execution time per tool |